In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler



In [5]:
# Define dataset path
data_dir = r"C:\Users\abdal\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset"

# Load data
X_train = pd.read_csv(os.path.join(data_dir, "train", "X_train.txt"), delim_whitespace=True, header=None)
X_test = pd.read_csv(os.path.join(data_dir, "test", "X_test.txt"), delim_whitespace=True, header=None)
y_train = pd.read_csv(os.path.join(data_dir, "train", "y_train.txt"), delim_whitespace=True, header=None)
y_test = pd.read_csv(os.path.join(data_dir, "test", "y_test.txt"), delim_whitespace=True, header=None)



In [6]:
# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input data for CNN-LSTM model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train.values.ravel())
y_test = label_encoder.transform(y_test.values.ravel())



In [7]:
# Convert labels to categorical
num_classes = len(np.unique(y_train))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)



In [10]:
# Build CNN-LSTM model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    LSTM(100, return_sequences=True),
    LSTM(100),
    
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))



Epoch 1/10
115/115 [==============================] - 60s 487ms/step - loss: 1.2475 - accuracy: 0.3779 - val_loss: 1.0296 - val_accuracy: 0.4917
Epoch 2/10
115/115 [==============================] - 48s 416ms/step - loss: 1.0174 - accuracy: 0.5079 - val_loss: 0.9054 - val_accuracy: 0.5640
Epoch 3/10
115/115 [==============================] - 59s 515ms/step - loss: 0.9141 - accuracy: 0.5646 - val_loss: 0.8234 - val_accuracy: 0.6037
Epoch 4/10
115/115 [==============================] - 57s 494ms/step - loss: 0.8467 - accuracy: 0.6027 - val_loss: 0.8586 - val_accuracy: 0.5969
Epoch 5/10
115/115 [==============================] - 51s 441ms/step - loss: 0.8068 - accuracy: 0.6240 - val_loss: 0.8120 - val_accuracy: 0.6040
Epoch 6/10
115/115 [==============================] - 50s 431ms/step - loss: 0.7795 - accuracy: 0.6428 - val_loss: 0.7644 - val_accuracy: 0.6457
Epoch 7/10
115/115 [==============================] - 49s 423ms/step - loss: 0.7555 - accuracy: 0.6474 - val_loss: 0.7407 - val_ac

In [11]:
# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


93/93 [==============================] - 5s 53ms/step - loss: 0.7323 - accuracy: 0.6461
Test Accuracy: 0.6461
